In [1]:
import emission.storage.timeseries.timequery as estt
import emission.storage.timeseries.abstract_timeseries as esta

In [2]:
import emission.core.get_database as edb
import bson.objectid as boi
from uuid import UUID

In [3]:
edb.get_timeseries_db().find_one({"_id": boi.ObjectId('57e44d88f6858f7387263233')})

In [4]:
tq = estt.TimeQuery("metadata.write_ts", 1466657016.201 - 100, 1466657016.201 + 100)
loc_df = esta.TimeSeries.get_time_series(UUID('96993ce8-e550-4535-aa28-836804596d92')).get_data_df("background/filtered_location", tq)

In [5]:
len(loc_df)

9

In [6]:
loc_df.ts.diff().tolist()

[nan,
 17.506999969482422,
 14.493000030517578,
 16.608999967575073,
 15.391000032424927,
 27.677999973297119,
 30.950000047683716,
 31.240000009536743,
 31.131999969482422]

In [7]:
to_del = loc_df[loc_df.ts.diff() < 0][["fmt_time", "_id"]]
print to_del.index, len(to_del)
to_del

Int64Index([], dtype='int64') 0


,fmt_time,_id


In [8]:
len(loc_df.drop(to_del.index.tolist()))

9

In [9]:
out_of_order_id_list = to_del["_id"].tolist()
edb.get_timeseries_db().remove({"_id": {"$in": out_of_order_id_list}})

{u'n': 0, u'ok': 1}